In [1]:
# ---------- visualizzo piu output a schermo insieme ------------------------------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
#-------------------------- scelgo se usare CPU o GPU ------------------------------
import tensorflow as tf
import numpy as np
import os

SEED = 1234
tf.random.set_seed(SEED)
cwd = os.getcwd()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
elif cpus:
    try:
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        print(e)
#-------------------------------------------------------------------------------------

1 Physical GPUs, 1 Logical GPUs


In [2]:
#---------------serve per dividere in sottocartelle il training set-------------------
import json, shutil, os

with open('MaskDataset\\train_gt.json', 'r') as labels:
    data=labels.read()
#----------- scegli cartella immagini miste ------------------
imm_source= os.path.join(cwd, 'MaskDataset\\training')
#----------- nome dataset organizzato ------------------------
data_dir=os.path.join(cwd,'MaskDataset\\dataset')
#----------- se non ce ordino tutto --------------------------
if not os.path.exists(data_dir):
    dir0 = os.path.join(data_dir, '0')
    dir1 = os.path.join(data_dir, '1')
    dir2 = os.path.join(data_dir, '2')
    os.mkdir(data_dir)
    os.mkdir(dir0)
    os.mkdir(dir1)
    os.mkdir(dir2)
    #leggo i label dal file json
    obj = json.loads(data)
    #ciclo su tutti i label e sposto immagini in cartelle separate
    for i in obj:
        if obj[i]==0:
            shutil.copy(os.path.join(imm_source,i), dir0,  follow_symlinks=True)
        elif obj[i]==1:
            shutil.copy(os.path.join(imm_source,i), dir1,  follow_symlinks=True)
        elif obj[i]==2:
            shutil.copy(os.path.join(imm_source,i), dir2,  follow_symlinks=True)

    labels.close()
    
#------------------------------------------------------------------------------------

In [3]:
#------------------------ creo i data generator -------------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# scegli se applicare data augmentation (solo su training)
apply_data_augmentation = True

#---------------- training ----------------------------------------
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(validation_split=0.2,
                                        rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range=0.15,
                                        shear_range=0.15,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='nearest',
                                        #cval=0,
                                        rescale=1./255)
# datagenerator senza augmentation
else:
    train_data_gen = ImageDataGenerator(validation_split=0.2,
                                        rescale=1./255)
    
#------------------- validation -----------------------------------   
validation_data_gen = ImageDataGenerator(validation_split=0.2,
                                         rescale=1./255)
#------------------------------------------------------------------

In [4]:
#----------------------------- Creo generators da file ------------------------------------
source_dir = os.path.join(cwd,'MaskDataset')

# Batch size
bs = 32

# dimensione immagini (VGG16 vuole 224x224)
img_h=299
img_w=299

#ho tre calssi: 0, 1, 2
num_classes=3

# usa se vuoi assegnare label diversi
decide_class_indices = False
if decide_class_indices:
    classes = ['0', # no maschera
               '1', # tutti maschera
               '2'] # alcuni maschera
else:
    classes=None

# Training set
dataset_dir = os.path.join(source_dir, 'dataset')
train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               target_size=(img_h, img_w),
                                               subset='training',
                                               batch_size=bs,
                                               classes=classes,
                                               color_mode="rgb",
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

# Validation set
validation_gen = validation_data_gen.flow_from_directory(dataset_dir,
                                                         target_size=(img_h, img_w),
                                                         subset='validation',
                                                         batch_size=bs,
                                                         classes=classes,
                                                         color_mode="rgb",
                                                         class_mode='categorical',
                                                         shuffle=False,
                                                         seed=SEED)

#---------------------------------------------------------------------------------------------

Found 4534 images belonging to 3 classes.
Found 1132 images belonging to 3 classes.


In [5]:
#------------------------ creo i data generator -------------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# scegli se applicare data augmentation (solo su training)
apply_data_augmentation = True

#---------------- training ----------------------------------------
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(validation_split=0.2,
                                        rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range=0.15,
                                        shear_range=0.15,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='nearest',
                                        #cval=0,
                                        rescale=1./255)
# datagenerator senza augmentation
else:
    train_data_gen = ImageDataGenerator(validation_split=0.2,
                                        rescale=1./255)
    
#------------------- validation -----------------------------------   
validation_data_gen = ImageDataGenerator(validation_split=0.2,
                                         rescale=1./255)
#------------------------------------------------------------------

In [6]:
#----------------------------- Creo generators da file ------------------------------------
source_dir = os.path.join(cwd,'MaskDataset')

# Batch size
bs = 32

# dimensione immagini (VGG16 vuole 224x224)
img_h=299
img_w=299

#ho tre calssi: 0, 1, 2
num_classes=3

# usa se vuoi assegnare label diversi
decide_class_indices = False
if decide_class_indices:
    classes = ['0', # no maschera
               '1', # tutti maschera
               '2'] # alcuni maschera
else:
    classes=None

# Training set
dataset_dir = os.path.join(source_dir, 'dataset')
train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               target_size=(img_h, img_w),
                                               subset='training',
                                               batch_size=bs,
                                               classes=classes,
                                               color_mode="rgb",
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

# Validation set
validation_gen = validation_data_gen.flow_from_directory(dataset_dir,
                                                         target_size=(img_h, img_w),
                                                         subset='validation',
                                                         batch_size=bs,
                                                         classes=classes,
                                                         color_mode="rgb",
                                                         class_mode='categorical',
                                                         shuffle=False,
                                                         seed=SEED)

#---------------------------------------------------------------------------------------------

Found 4534 images belonging to 3 classes.
Found 1132 images belonging to 3 classes.


In [7]:
#---------------------- Creo dataset training -------------------------------------------------------------------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

#---------------------- Creo dataset validation -----------------------------------------------------------------
valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()


In [ ]:
list(train_dataset)

In [1]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    
    # extract the bounding box from the first face
    faces = []
    for face in results:
        x1, y1, width, height = face['box']
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        faces.append(face_array)
    
    return faces
 
# load the photo and extract the face
pixels = extract_face('MaskDataset\\dataset\\0\\10009.jpg')
# plot the extracted face
pyplot.imshow(pixels[3])
# show the plot
pyplot.show()

Using TensorFlow backend.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at c:\users\customer\anaconda3\envs\tf_env\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_791]

Function call stack:
keras_scratch_graph
